 ## Project 2
 ### Team:4
    
    Anji Asthana
    Eugene Witherspoon
    Fatma Butun
    Matt Keeley
    Shay Masood


 

## Overview 

####  The purpose of this project is to examine the distribution of fire and rescue incident calls across different neighborhoods in Cincinnati, OH and inform residents about the fire department’s performance, including: 
#####   a. Ambulance headquarters placement 
#####   b. Fire hydrant pressure adjustments
#####   c. Allocation of fire fighter/rescue resources
#####   d. Helping high-risk citizens stay informed about response time
#####   e. Serve as a resource for fire department managers/administration


In [1]:
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine



## Extract the Data 

### This part is using the original data source to obtain a file that is smaller in size 

### The code here is commented out because the original file is not available on github

In [2]:
# # read the csv into a dataframe
# cincinnati_fire = pd.read_csv("Cincinnati_Fire_Incidents__CAD___including_EMS__ALS_BLS_.csv")

In [3]:
# cincinnati_fire.tail()

In [4]:
# # Check out how many rows and columns are in the dataframe

# cincinnati_fire.shape

## Transform and Clean the Data

### Reduce the original Data

In [5]:
# # check the column types
# cincinnati_fire.dtypes

In [6]:
# # Change the CREATE_TIME_INCIDENT  column into datetime64 in order to do further analysis

# cincinnati_fire["CREATE_TIME_INCIDENT"] = pd.to_datetime(cincinnati_fire["CREATE_TIME_INCIDENT"])


In [7]:
# # check out the types of the columns to see if the transformation has worked
# cincinnati_fire.dtypes

In [8]:
# # remove the rows with incidents before the year 2019

# cincinnati_filtered_df = cincinnati_fire.loc[(cincinnati_fire['CREATE_TIME_INCIDENT'] >= '01/01/2019')]
# cincinnati_filtered_df.shape  

In [9]:
# # save the filtered dataframe and use this as your source data

# cincinnati_filtered_df.to_csv('cincinnati_filtered_df.csv', index=False)

## Load the filtered data for the analysis


In [10]:
# read the csv into a df

cincinnati_fire_filtered = pd.read_csv("cincinnati_filtered_df.csv")

In [11]:
# Change the CREATE_TIME_INCIDENT  column into datetime64 in order to do further analysis

cincinnati_fire_filtered["CREATE_TIME_INCIDENT"] = pd.to_datetime(cincinnati_fire_filtered["CREATE_TIME_INCIDENT"])

In [12]:
# Change the ARRIVAL_TIME_PRIMARY_UNIT column into datetime64 in order to do further analysis

cincinnati_fire_filtered["ARRIVAL_TIME_PRIMARY_UNIT"] = pd.to_datetime(cincinnati_fire_filtered["ARRIVAL_TIME_PRIMARY_UNIT"])

In [13]:
# Change the CLOSED_TIME_INCIDENT column into datetime64 in order to do further analysis

cincinnati_fire_filtered["CLOSED_TIME_INCIDENT"] = pd.to_datetime(cincinnati_fire_filtered["CLOSED_TIME_INCIDENT"])
cincinnati_fire_filtered.dtypes

ADDRESS_X                                 object
LATITUDE_X                               float64
LONGITUDE_X                              float64
AGENCY                                    object
CREATE_TIME_INCIDENT              datetime64[ns]
DISPOSITION_TEXT                          object
EVENT_NUMBER                              object
INCIDENT_TYPE_ID                          object
INCIDENT_TYPE_DESC                        object
NEIGHBORHOOD                              object
ARRIVAL_TIME_PRIMARY_UNIT         datetime64[ns]
BEAT                                      object
CLOSED_TIME_INCIDENT              datetime64[ns]
DISPATCH_TIME_PRIMARY_UNIT                object
CFD_INCIDENT_TYPE                         object
CFD_INCIDENT_TYPE_GROUP                   object
COMMUNITY_COUNCIL_NEIGHBORHOOD            object
dtype: object

In [14]:
# remove rows with at least 1 NaN

cincinnati_fire_filtered = cincinnati_fire_filtered.dropna()
cincinnati_fire_filtered.shape

(127528, 17)

In [15]:
# Calculate the duration for the arrival after the incident was reported

cincinnati_fire_filtered["ARRIVAL_DURATION"] = (cincinnati_fire_filtered["ARRIVAL_TIME_PRIMARY_UNIT"] - cincinnati_fire_filtered["CREATE_TIME_INCIDENT"]).astype('timedelta64[m]')



In [16]:
# Drop the rows where the arrival duration column is less than 0 and more than 100. (response time cannot be negative and )
cincinnati_fire_filtered = cincinnati_fire_filtered.drop(cincinnati_fire_filtered[cincinnati_fire_filtered["ARRIVAL_DURATION"] < 0 ].index)
cincinnati_fire_filtered.shape
cincinnati_fire_filtered = cincinnati_fire_filtered.drop(cincinnati_fire_filtered[cincinnati_fire_filtered["ARRIVAL_DURATION"]  > 100 ].index)
cincinnati_fire_filtered.shape

(126258, 18)

In [17]:
# Calculate the duration for closing the incident after the incident was reported

cincinnati_fire_filtered["CLOSING_DURATION"] = (cincinnati_fire_filtered["CLOSED_TIME_INCIDENT"] - cincinnati_fire_filtered["CREATE_TIME_INCIDENT"]).astype('timedelta64[m]')



In [18]:
# cincinnati_fire_filtered.head()

In [19]:
# drop dupliacte rows and sort df by create time incident column
cincinnati_fire_filtered = cincinnati_fire_filtered.drop_duplicates()
cincinnati_fire_filtered = cincinnati_fire_filtered.sort_values(by=["CREATE_TIME_INCIDENT"])



In [20]:
cincinnati_fire_filtered[cincinnati_fire_filtered['NEIGHBORHOOD']=='WALNUT HILLS']

,ADDRESS_X,LATITUDE_X,LONGITUDE_X,AGENCY,CREATE_TIME_INCIDENT,DISPOSITION_TEXT,EVENT_NUMBER,INCIDENT_TYPE_ID,INCIDENT_TYPE_DESC,NEIGHBORHOOD,ARRIVAL_TIME_PRIMARY_UNIT,BEAT,CLOSED_TIME_INCIDENT,DISPATCH_TIME_PRIMARY_UNIT,CFD_INCIDENT_TYPE,CFD_INCIDENT_TYPE_GROUP,COMMUNITY_COUNCIL_NEIGHBORHOOD,ARRIVAL_DURATION,CLOSING_DURATION
262370,IOWA AV,39.123646,-84.499822,CFD,2019-01-01 04:48:00,MEDT: MEDIC TRANSPORT,CFD190101000168,10D4,CLAMMY OR COLD SWEATS,WALNUT HILLS,2019-01-01 04:53:00,ST19,2019-01-01 05:33:00,1/1/2019 4:49,ALS,CHEST PAIN / CHEST DISCOMFORT (NON-TRAUMATIC),WALNUT HILLS,5.0,45.0
264156,LINCOLN AV,39.131525,-84.479608,CFD,2019-01-01 05:47:00,MEDT: MEDIC TRANSPORT,CFD190101000177,28C5,SUDDEN PARALYSIS OR FACIAL DROOP (ONE SIDE),WALNUT HILLS,2019-01-01 05:50:00,ST23,2019-01-01 06:21:00,1/1/2019 5:47,ALS,STROKE (CVA) / TRANSIENT ISCHEMIC ATTACK (TIA),WALNUT HILLS,3.0,34.0
264154,VICTORY PKWY,39.127638,-84.485200,CFD,2019-01-01 06:05:00,MEDT: MEDIC TRANSPORT,CFD190101000179,26A1,NO PRIORITY SYMPTOMS (COMPLAINT CONDITIONS 2–1...,WALNUT HILLS,2019-01-01 06:07:00,ST23,2019-01-01 06:50:00,1/1/2019 6:06,BLS,SICK PERSON,WALNUT HILLS,2.0,45.0
262444,CONCORD ST,39.123490,-84.494040,CFD,2019-01-01 07:42:00,EMS: NO TRANSPORT,CFD190101000094,13C1,NOT ALERT,WALNUT HILLS,2019-01-01 07:45:00,ST23,2019-01-01 07:56:00,1/1/2019 7:43,ALS,DIABETIC PROBLEMS,WALNUT HILLS,3.0,14.0
264106,PARK AV,39.127636,-84.486384,CFD,2019-01-01 10:02:00,MEDD: MT DISREGARDED,CFD190101000227,19D1,NOT ALERT,WALNUT HILLS,2019-01-01 10:05:00,ST23,2019-01-01 10:06:00,1/1/2019 10:03,ALS,HEART PROBLEM / A.I.C.D,WALNUT HILLS,3.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271691,GILBERT AV,39.128556,-84.489953,CFD,2021-12-22 02:41:00,MEDT: MEDIC TRANSPORT,CFD211222000104,12A1,NOT SEIZING NOW AND EFFECTIVE BREATHING VERIFI...,WALNUT HILLS,2021-12-22 02:44:00,ST23,2021-12-22 03:16:00,12/22/2021 2:42,BLS,CONVULSIONS / SEIZURES,WALNUT HILLS,3.0,35.0
272143,BEECHER ST,39.133423,-84.489272,CFD,2021-12-22 05:04:00,MED: MT RESPONSE NO TRANSPORT,CFD211222000017,32B2,MEDICAL ALARM (ALERT) NOTIFICATIONS (NO PATIEN...,WALNUT HILLS,2021-12-22 05:08:00,ST23,2021-12-22 05:20:00,12/22/2021 5:05,ALS,UNKNOWN PROBLEM (PERSON DOWN),WALNUT HILLS,4.0,16.0
271648,BEECHER ST,39.133593,-84.489902,CFD,2021-12-22 05:08:00,MEDT: MEDIC TRANSPORT,CFD211222000147,21B1,POSSIBLY DANGEROUS HEMORRHAGE,WALNUT HILLS,2021-12-22 05:11:00,ST23,2021-12-22 05:45:00,12/22/2021 5:08,ALS,HEMORRHAGE / LACERATIONS,WALNUT HILLS,3.0,37.0
272129,BEECHER ST,39.133743,-84.489152,CFD,2021-12-22 07:23:00,MEDT: MEDIC TRANSPORT,CFD211222000031,31D4,CHANGING COLOR,WALNUT HILLS,2021-12-22 07:26:00,ST23,2021-12-22 08:33:00,12/22/2021 7:23,ALS,UNCONSCIOUS / FAINTING (NEAR),WALNUT HILLS,3.0,70.0


In [21]:
# prepare 3 new dataframes with the required columns only 
    
# 1-prepare an incidents vs neighborhood df
    
neighborhood_incidents = cincinnati_fire_filtered [["EVENT_NUMBER", "NEIGHBORHOOD", "LATITUDE_X", "LONGITUDE_X", "INCIDENT_TYPE_DESC"]]
neighborhood_incidents.head()

,EVENT_NUMBER,NEIGHBORHOOD,LATITUDE_X,LONGITUDE_X,INCIDENT_TYPE_DESC
262646,CFD190101000027,CAMP WASHINGTON,39.135839,-84.536683,ABDOMINAL PAIN/CRAMPING (< 6 MONTHS/24 WEEKS A...
262394,CFD190101000144,ROSELAWN,39.189808,-84.458955,PUBLIC ASSIST (NO INJURIES AND NO PRIORITY SYM...
262640,CFD190101000033,ROSELAWN,39.197923,-84.459645,PUBLIC ASSIST (NO INJURIES AND NO PRIORITY SYM...
262639,CFD190101000034,HARTWELL,39.216054,-84.464857,UNKNOWN STATUS/OTHER CODES NOT APPLICABLE
262392,CFD190101000146,EVANSTON,39.143939,-84.458513,INEFFECTIVE BREATHING


In [22]:
# 2- prepare an incidents vs arrival time df

incidents_time_duration = cincinnati_fire_filtered [["EVENT_NUMBER", "ARRIVAL_DURATION", "CREATE_TIME_INCIDENT", "NEIGHBORHOOD", "INCIDENT_TYPE_DESC"]]
incidents_time_duration.head(15)

,EVENT_NUMBER,ARRIVAL_DURATION,CREATE_TIME_INCIDENT,NEIGHBORHOOD,INCIDENT_TYPE_DESC
262646,CFD190101000027,2.0,2019-01-01 01:08:00,CAMP WASHINGTON,ABDOMINAL PAIN/CRAMPING (< 6 MONTHS/24 WEEKS A...
262394,CFD190101000144,5.0,2019-01-01 01:27:00,ROSELAWN,PUBLIC ASSIST (NO INJURIES AND NO PRIORITY SYM...
262640,CFD190101000033,14.0,2019-01-01 01:31:00,ROSELAWN,PUBLIC ASSIST (NO INJURIES AND NO PRIORITY SYM...
262639,CFD190101000034,8.0,2019-01-01 01:32:00,HARTWELL,UNKNOWN STATUS/OTHER CODES NOT APPLICABLE
262392,CFD190101000146,5.0,2019-01-01 01:46:00,EVANSTON,INEFFECTIVE BREATHING
262501,CFD190101000037,5.0,2019-01-01 01:50:00,WESTWOOD,ALTERED LEVEL OF CONSCIOUSNESS
262500,CFD190101000038,4.0,2019-01-01 01:52:00,DOWNTOWN,"STANDING, SITTING, MOVING, OR TALKING"
262498,CFD190101000040,5.0,2019-01-01 01:53:00,OAKLEY,CONTINUOUS OR MULTIPLE SEIZURES
262499,CFD190101000039,6.0,2019-01-01 01:53:00,CLIFTON,NOT DANGEROUS BODY AREA
262497,CFD190101000041,9.0,2019-01-01 02:01:00,MT. AIRY,DIFFICULTY SPEAKING BETWEEN BREATHS


In [23]:
# 3- prepare a grouped neighborhood dataframe

neighborhood_incidents["NEIGHBORHOOD"].unique()
neighborhood_incidents_grouped = neighborhood_incidents.groupby(["NEIGHBORHOOD", "INCIDENT_TYPE_DESC"]).size().reset_index(name='counts')
neighborhood_incidents_grouped.head(15)

,NEIGHBORHOOD,INCIDENT_TYPE_DESC,counts
0,AVONDALE,1ST TRIMESTER HEMORRHAGE OR MISCARRIAGE,7
1,AVONDALE,1ST TRIMESTER SERIOUS HEMORRHAGE,9
2,AVONDALE,2ND TRIMESTER HEMORRHAGE OR MISCARRIAGE,8
3,AVONDALE,3RD TRIMESTER HEMORRHAGE,6
4,AVONDALE,ABDOMINAL PAIN,219
5,AVONDALE,ABDOMINAL PAIN/CRAMPING (< 6 MONTHS/24 WEEKS A...,25
6,AVONDALE,ABNORMAL BEHAVIOR,33
7,AVONDALE,ABNORMAL BREATHING,775
8,AVONDALE,ABNORMAL BREATHING (PARTIAL OBSTRUCTION),23
9,AVONDALE,ABNORMAL BREATHING WITH MULTIPLE FLU SYMPTOMS,25


In [24]:
# put similar incidents under the same incident type description. 

# what can we put together?

## Create Connection to SQL database and load the tables

In [25]:
# create a connection string to postgres
connection_string = "postgres:bootcamp@localhost:5432/cincinnatifire"
engine = create_engine(f'postgresql://{connection_string}')

In [26]:
# check the tables in the cincinnatifire database
engine.table_names()

<ipython-input-26-fae4aa58819e>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['neighborhood_incidents_grouped',
 'neighborhood_incidents',
 'incidents_time_duration']

In [27]:
# Insert df data to the tables
neighborhood_incidents.to_sql(name='neighborhood_incidents', con=engine, if_exists='append', index=False)



In [28]:
incidents_time_duration.to_sql(name='incidents_time_duration', con=engine, if_exists='append', index=False)

In [29]:
neighborhood_incidents_grouped.to_sql(name='neighborhood_incidents_grouped', con=engine, if_exists='append', index=False)

In [30]:
# check if the tables are correctly loaded into the database

pd.read_sql_query('select * from incidents_time_duration', con=engine).head()

,EVENT_NUMBER,ARRIVAL_DURATION,CREATE_TIME_INCIDENT,NEIGHBORHOOD,INCIDENT_TYPE_DESC
0,CFD190101000027,2.0,2019-01-01 01:08:00,CAMP WASHINGTON,ABDOMINAL PAIN/CRAMPING (< 6 MONTHS/24 WEEKS A...
1,CFD190101000144,5.0,2019-01-01 01:27:00,ROSELAWN,PUBLIC ASSIST (NO INJURIES AND NO PRIORITY SYM...
2,CFD190101000033,14.0,2019-01-01 01:31:00,ROSELAWN,PUBLIC ASSIST (NO INJURIES AND NO PRIORITY SYM...
3,CFD190101000034,8.0,2019-01-01 01:32:00,HARTWELL,UNKNOWN STATUS/OTHER CODES NOT APPLICABLE
4,CFD190101000146,5.0,2019-01-01 01:46:00,EVANSTON,INEFFECTIVE BREATHING
